In [12]:
import os
import dotenv

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

In [13]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage
from llama_index.core import Settings
import os

llm = OpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)

embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    api_key=os.getenv("OPENAI_API_KEY"),
    api_base=os.getenv("OPENAI_BASE_URL"),
)

Settings.embed_model = embed_model

In [14]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./local_index"
    )
    index = load_index_from_storage(storage_context)
except:
    # Note: reduce batch size to stay within token limits
    documents = SimpleDirectoryReader("./Big Star Collectibles").load_data()
    index = VectorStoreIndex.from_documents(documents, insert_batch_size=150)
    index.storage_context.persist(persist_dir="./local_index")


In [18]:
query_engine = index.as_query_engine(
  llm=llm
)
response = query_engine.query("When did the Big Star Collectibles Story Start?")
print(response.response)

The Big Star Collectibles story began in 2013 at the International Arts Conference.


In [19]:
response = query_engine.query("Who started Big Star Collectibles?")
print(response.response)

Big Star Collectibles was started by Saura Chen.
